In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/MyDrive/instruct-caption')

Mounted at /content/gdrive


In [ ]:
!pip install -q transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import json

In [ ]:
peft_model_id = "saved_model"
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen1.5-1.8B-Chat",
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen1.5-1.8B-Chat")

model.load_adapter(peft_model_id)
path = "dataset.jsonl"
with open(path, "r") as f:
  lst = [json.loads(line) for line in f.readlines()]
  correct = 0
  total = len(lst)
  for data in lst:
    user_prompt = data['input']
    system_prompt = data['instruction']

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    if response == data['output']:
      correct+=1

print(f"Prediction accuracy is {correct/total}")